# Traffic Tune - Optimizing Traffic Signals with Reinforcement Learning

## Introduction

Welcome to the Traffic Tune POC notebook. Our project focused on optimizing traffic signal control using reinforcement learning. Traffic congestion is a major problem in urban areas, leading to increased travel times, fuel consumption, and pollution. Traditional traffic signal control systems often struggle to adapt to dynamic traffic conditions, resulting in suboptimal traffic flow.

Traffic Tune is a recommendation system that leverages reinforcement learning to dynamically adjust traffic signals at intersections. By learning from traffic patterns in real-time, Traffic Tune aims to improve traffic flow, reduce congestion, and enhance overall transportation efficiency.

In this POC, we will demonstrate how to train a reinforcement learning agent to optimize traffic signal control in a simulated environment. We will use the SUMO (Simulation of Urban MObility) traffic simulation tool and the Stable Baselines3 library to train a Deep Q-Network (DQN) agent to learn an optimal traffic signal control policy.


# Setup and Installations

In [1]:
import env_manager as env_manager
import ppo_trainer as ppo_trainer

In [2]:
def first_link_train(num_intersection: int, experiment_type: str, ppo_config: str, env_config: str, num_routes_files: int):
    running_result = []
    # Initialize the environment manager
    manager = env_manager.EnvManager(f"{experiment_type}Environment", env_config,
                                     json_id=f"intersection_{num_intersection}")
    generator = manager.env_generator(
        f"Nets/intersection_{num_intersection}/route_xml_path_intersection_{num_intersection}.txt")
    rou, csv = next(generator)
    env_kwargs = manager.initialize_env(rou, csv)

    print(f"\nEnv creat for intersection_{num_intersection}",
          "\nNet path:", manager.kwargs["net_file"],
          "\nRoute path:", rou,
          "\nCsv path:", csv)

    # Initialize the PPO agent
    ppo_agent = ppo_trainer.PPOTrainer(ppo_config, manager, experiment_type=experiment_type)
    ppo_agent.build_config()

    # Train the PPO agent
    result = ppo_agent.train()
    running_result.append(result)
    
    if num_routes_files != 0:
        for i in range(num_routes_files-1):
            chain_result = chain_training(manager, generator, ppo_agent, running_result)
            if chain_result is not None:
                running_result.append(chain_result)
    
    return running_result

def chain_training(manager: env_manager, generator: env_manager.EnvManager.env_generator, ppo_agent: ppo_trainer,
                   running_result: list):
    # take the best config from the previous training 
    best = running_result[-1].get_best_result("env_runners/episode_reward_max", "max")

    # Initialize the environment manager with new route file
    rou, csv = next(generator)
    env_kwargs = manager.initialize_env(rou, csv)
    
    print(f"\nEnv creat",
          "\nNet path:", manager.kwargs["net_file"],
          "\nRoute path:", rou,
          "\nCsv path:", csv)
    
    # continue the training with the best config
    ppo_agent.config = best.config
    ppo_agent.build_config()
    
    result = ppo_agent.train()
    return result

In [3]:
num_intersection_to_train = 6  # Choose which intersection you want to train

experiment_type = "SingleAgent"  # Choose the experiment_type: SingleAgent | MultiAgent

num_routes_to_train = 2

env_config_file_path = "env_config.json"

ppo_config_file_path = "ppo_config.json"

In [ ]:
results = first_link_train(num_intersection=num_intersection_to_train, experiment_type=experiment_type, ppo_config=ppo_config_file_path, env_config=env_config_file_path, num_routes_files=num_routes_to_train)

In [5]:
print(results)

[ResultGrid<[
  Result(
    metrics={'evaluation': {'env_runners': {'episode_reward_max': -590.59, 'episode_reward_min': -640.47, 'episode_reward_mean': -613.62, 'episode_len_mean': 720.0, 'episode_media': {}, 'episodes_timesteps_total': 2160, 'policy_reward_min': {}, 'policy_reward_max': {}, 'policy_reward_mean': {}, 'custom_metrics': {}, 'hist_stats': {'episode_reward': [-590.59, -609.8, -640.47], 'episode_lengths': [720, 720, 720]}, 'sampler_perf': {'mean_raw_obs_processing_ms': 1.6092773918915897, 'mean_inference_ms': 0.40721189213678693, 'mean_action_processing_ms': 0.04515549800295456, 'mean_env_wait_ms': 27.777304401711945, 'mean_env_render_ms': 0.0}, 'num_faulty_episodes': 0, 'connector_metrics': {'ObsPreprocessorConnector_ms': 0.0037590662638346353, 'StateBufferConnector_ms': 0.0024477640787760415, 'ViewRequirementAgentConnector_ms': 0.05824565887451172}, 'num_episodes': 3, 'episode_return_max': -590.59, 'episode_return_min': -640.47, 'episode_return_mean': -613.62, 'episodes_